## Build the Action-BodyShape atlas based on BABEL

### relative packages and paths

In [4]:
import numpy as np
import os
import json
import matplotlib.pyplot as plt

%matplotlib inline

featurePath = "/home/kxue/Work/MotionGen/AMASS/"
dataPath = "/home/kxue/Work/MotionGen/babel_v1.0_release/"

files = [
    "extra_train.json", # 7921 objs
    "extra_val.json", # 2636 objs
    "test.json", # 2084
    "train.json", # 6615
    "val.json", #2193
]

### load label data

In [5]:
datas = []
for file in files:
    f = open(dataPath + file)
    datas.append(json.load(f))

exTrain, exVal, test, train, val = datas

print(
    "Size of train: %d, val: %d, test: %d, extra train: %d, extra val: %d" % 
    (len(train.keys()), len(val.keys()), len(test.keys()), len(exTrain.keys()), len(exVal.keys()))
)

Size of train: 6615, val: 2193, test: 2084, extra train: 7921, extra val: 2636


#### check the extra label

In [7]:
keyExTrain = exTrain.keys()
counter = 0
for key in keyExTrain:
    if exTrain[key]["frame_anns"]:
        counter += 1
print(
    "there are %d of %d sequences in extra trainset have frame level annotation"
    % (counter, len(exTrain.keys()))
)

there are 47 of 7921 sequences in extra trainset have frame level annotation


#### check the original label

##### raw_label - proc_label

In [6]:
keyTrain = train.keys()
keyVal = val.keys()
keyTest = test.keys()

# build a dict of proc_label-raw_label
procTrans = {}
for key in keyTrain:
    objLabels = train[key]["seq_ann"]["labels"]
    for label in objLabels:
        if label["raw_label"]!=label["proc_label"]:
            if label["proc_label"] not in procTrans.keys():
                procTrans[label["proc_label"]] = set()
            procTrans[label["proc_label"]].add(label["raw_label"])
procTrans

{'step': {'step ', 'stepping'},
 'tpose': {'T-Pose', 'T-pose', 'T-posing', 't-pose', 't-pose '},
 'stand': {'Stand', 'Standing', 'standing', 'standing '},
 'walk backwards': {'Walking backwards', 'walking backwards'},
 'low head': {'lower head'},
 'kick feet': {'kick feet '},
 'jump jacks': {'Jumping Jacks', 'Jumping jacks', 'jumping jacks'},
 'skip': {'skipping', 'skipping '},
 'walk': {'Walk',
  'Walk ',
  'Walking',
  'Walking ',
  'Walks',
  'walk ',
  'walking',
  'walking ',
  'walking  ',
  'walking.',
  'walks',
  'walks '},
 'bend over': {'Bending over', 'Bending over.', 'bending over'},
 'low arms': {'lower arms'},
 'walk in place': {'Walking in place.', 'walking in place'},
 'jog in place': {'Jogging in place', 'jogging in place'},
 'run in place': {'Running in place.', 'running in place'},
 'crouch walk': {'Crouch walking', 'crouch walking'},
 'wakle backwards': {'wakling backwards'},
 'stand back up': {'standing back up'},
 'turn around': {'Turn around', 'turn around ', 't

##### actions - proc_label

In [7]:
# build a dict of action-proc_label
actProclabel = {}
for key in keyTrain:
    objLabels = train[key]["seq_ann"]["labels"]
    for label in objLabels:
        if label["act_cat"]:
            for action in label["act_cat"]:
                if action not in actProclabel.keys():
                    actProclabel[action] = set()
                actProclabel[action].add(label["proc_label"])
actProclabel

{'walk': {'assist walk',
  'backwards walk',
  'balance walk',
  'catwalk',
  'crouch walk',
  'drunk walking',
  'hobble',
  'just walk around',
  'just walk in a circle',
  'moonwalk',
  'pace',
  'pace back and forth',
  'pace forward',
  'poor posture walk',
  'power walk',
  'push forward',
  'saunt',
  'shuffle back and forth',
  'shuffle to the side',
  'side walk',
  'slow down',
  'slow walk',
  'slow walking',
  'slowly walk',
  'speed walk',
  'stagger',
  'stairs walk',
  'stand walk',
  'stride',
  'stroll',
  'strut',
  'strut forward',
  'swagger',
  'take a walk',
  'take three steps forward',
  'up with walk',
  'waddle',
  'wade',
  'walk',
  'walk across a beam',
  'walk across balance beam',
  'walk and bend down',
  'walk and jump',
  'walk and trip',
  'walk and turn',
  'walk and turn right',
  'walk around',
  'walk around in a circle',
  'walk around in a circle counterclockwise',
  'walk around in circles',
  'walk away',
  'walk back',
  'walk back and forth'

##### proc_label-actions

In [17]:
# build a dict of proc_label-action
proclabelAct = {}
for key in keyTrain:
    objLabels = train[key]["seq_ann"]["labels"]
    for label in objLabels:
        if label["proc_label"]:
            if label["proc_label"] not in proclabelAct.keys():
                proclabelAct[label["proc_label"]] = set()
            if label["act_cat"]:
                for act in label["act_cat"]:
                    proclabelAct[label["proc_label"]].add(act)
proclabelAct

{'walk': {'walk'},
 'scoop': {'fill', 'lift something'},
 'place': {'place something'},
 'turn': {'turn'},
 'bow': {'bow', 'lowering body part'},
 'dance': {'dance', 'perform'},
 'stop': {'stop'},
 'jump': {'jump'},
 't pose': {'t pose'},
 'spin': {'spin'},
 'toss': {'interact with/use object'},
 'catch': {'catch'},
 'throw': {'throw'},
 'backstep': {'backwards movement', 'step'},
 'sit': {'sit'},
 'clap': {'clap', 'hand movements', 'touching body part'},
 'arms down': {'arm movements', 'lowering body part'},
 'rub wrists': set(),
 'step': {'step'},
 'look around': {'look'},
 'wave arms': {'arm movements', 'gesture', 'wave'},
 'throw arms up': {'arm movements', 'raising body part'},
 'look back': {'look'},
 'kneel down': {'kneel'},
 'stand up': {'stand up'},
 'turn around': {'turn'},
 'hang': {'hang'},
 'shrug': {'shrug'},
 'knock': {'knock'},
 'tpose': {'t pose'},
 'stand': {'stand'},
 'walk backwards': {'backwards movement', 'walk'},
 'walk forward': {'walk'},
 'move left arm from le

##### shape-actions

In [8]:
shapeActDict = {}
actShapeDict = {}

def buildAtlas(data):
    # path to read shape and pose parameters
    featureData = np.load(featurePath + data["feat_p"])
    shape = tuple(featureData.f.betas)
    actCat = []
    # some sequence may only have sequence annotation
    if data["frame_ann"]:
        labels = data["frame_ann"]["labels"]
    else:
        labels = data["seq_ann"]["labels"]
    for label in labels:
        if label["act_cat"]:
            actCat = actCat + label["act_cat"]

    if shape not in shapeActDict.keys():
        shapeActDict[shape] = set()
    for act in actCat:
        shapeActDict[shape].add(act)
        if act not in actShapeDict.keys():
            actShapeDict[act] = set()
        actShapeDict[act].add(shape)

for key in keyTrain:
    data = train[key]
    buildAtlas(data)

for key in keyTest:
    data = test[key]
    buildAtlas(data)

for key in keyVal:
    data = val[key]
    buildAtlas(data)

##### number of actions and shapes

In [9]:
print("Totally, there are %d action categories" % len(actShapeDict.keys()))
print("Totally, there are %d shapes" % len(shapeActDict.keys()))

Totally, there are 243 action categories
Totally, there are 440 shapes


##### act shape distribution

In [13]:
actKey = actShapeDict.keys()
shapeNum = []
for act in actKey:
    shapeNum.append(len(actShapeDict[act]))
sortedIndex = np.argsort(shapeNum)
print("Top 50 actions with the most of shapes:")
print(np.array(list(actKey))[sortedIndex[-50:]][::-1])
print(np.sort(shapeNum)[::-1])

Top 50 actions with the most of shapes:
['transition' 'stand' 'walk' 'hand movements' 'arm movements' 'turn'
 'interact with/use object' 'backwards movement' 'raising body part'
 'jump' 'look' 'stand up' 'sit' 'step' 'stretch' 'run' 'forward movement'
 'bend' 'leg movements' 'touching body part' 'circular movement'
 'take/pick something up' 'throw' 'kick' 'sideways movement' 'jog'
 'foot movements' 'touch object' 'place something' 'head movements'
 't pose' 'waist movements' 'gesture' 'lowering body part'
 'exercise/training' 'wave' 'lift something' 'greet' 'grasp object'
 'action with ball' 'dance' 'squat' 'poses' 'knee movement' 'catch' 'lean'
 'swing body part' 'knock' 'move something' 'clap']
[411 378 339 310 303 284 260 253 247 235 221 215 214 205 200 189 184 183
 179 176 171 167 165 165 163 163 161 161 157 154 147 143 143 129 125 124
 112 107 107 107 104 100  99  96  96  89  87  85  84  82  81  79  79  76
  71  70  67  67  67  62  60  56  54  54  50  50  48  48  44  39  39  38
  

##### shape act distribution

In [35]:
shapeKey = shapeActDict.keys()
actNum = []
for act in shapeKey:
    actNum.append(len(shapeActDict[act]))
print(np.sort(actNum)[::-1])

[101  99  94  90  86  84  77  76  73  72  72  68  67  66  65  62  60  57
  55  55  54  51  49  49  49  48  48  48  47  47  47  46  45  44  44  43
  43  43  43  42  42  42  42  42  42  42  41  41  41  41  41  41  41  41
  40  40  40  40  40  39  39  39  39  39  39  38  38  38  38  37  37  37
  37  37  37  37  37  37  36  36  36  36  36  36  36  36  36  36  35  35
  35  35  35  35  35  35  35  35  35  34  34  34  34  34  34  34  34  34
  34  34  34  33  33  33  33  33  33  33  33  33  32  32  32  32  32  32
  32  32  32  31  31  31  31  31  31  31  31  31  31  31  31  31  31  30
  30  30  30  30  30  30  30  30  30  30  30  30  30  30  29  29  29  29
  29  29  29  29  29  29  29  28  28  28  28  28  28  28  28  28  28  28
  28  27  27  27  27  27  27  27  27  27  27  27  27  27  27  27  27  26
  26  26  26  26  26  26  26  26  26  26  26  26  26  26  26  26  26  26
  26  25  25  25  25  25  25  25  25  25  25  25  25  25  25  25  25  25
  25  24  24  24  24  24  24  24  24  24  24  24  2

##### summary

In [38]:
# scatter plot person(X)-action(Y)
x, y = [], []
sortedAct = np.array(list(actKey))[sortedIndex][::-1]
# from person with most actions
personIndex = np.argsort(actNum)[::-1]
for idxX, idx in enumerate(personIndex):
    actions = shapeActDict[list(shapeKey)[idx]]
    for action in actions:
        idxY = np.where(sortedAct==action)[0][0]
        x.append(idxX)
        y.append(idxY)



In [44]:
fig, ax = plt.subplots(figsize=(100, 100))
ax.scatter(x, y, s=4)
ax.set_yticks(np.arange(len(sortedAct)), sortedAct)
plt.show()